#MAY NEED TO INSTALL THE FOLLOWING LIBRARIES IN YOUR Google Cloud ENVIRONMENT 
pip install pandas pymysql flask prophet


In [4]:
from flask import Flask, request, jsonify
import pymysql
import pandas as pd
from prophet import Prophet

app = Flask(__name__)

# Function to connect to Cloud SQL
def db_connection():
    return pymysql.connect(
        host="your-cloudsql-ip",
        user="your-user",
        password="your-password",
        database="your-database",
        cursorclass=pymysql.cursors.DictCursor  # 辞書系で返す
    ) # GoogleCloudで設定した名前・ユーザーなどに変える

# Function to fetch historical sales data
def get_sales_data(product_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Fetch past sales for the given product
    cursor.execute("""
        USE product_management
        SELECT date, quantity  #MySQLに日にちのコラムを足す
        FROM stock_transaction
        WHERE product_id = %s
        ORDER BY date 
    """, (product_id,))
    
    sales = cursor.fetchall()
    conn.close()
    
    # Convert to Pandas DataFrame
    df = pd.DataFrame(sales)
    df.rename(columns={"date": "ds", "quantity": "y"}, inplace=True)
    
    return df

# Function to forecast next week's order quantity
def forecast_next_order(product_id):
    # Get sales data
    df = get_sales_data(product_id)
    
    if df.empty:
        return None  # No sales data available
    
    # Initialize Prophet model
    model = Prophet()
    model.fit(df)

    # Predict for the next 7 days
    future = model.make_future_dataframe(periods=7)
    forecast = model.predict(future)

    # Get next week's predicted sales
    next_week_forecast = forecast.tail(7)["yhat"].sum()
    
    # Apply a safety stock factor (adjust as needed)
    suggested_order_qty = round(next_week_forecast * 1.2)

    return suggested_order_qty

# API Endpoint for AppSheet
@app.route("/forecast-order", methods=["POST"])
def forecast_order():
    data = request.json
    product_id = data.get("product_id")

    if not product_id:
        return jsonify({"error": "Product ID is required"}), 400

    # Run forecast
    suggested_order = forecast_next_order(product_id)

    if suggested_order is None:
        return jsonify({"error": "No sales data available for this product"}), 404

    # Update Cloud SQL with the forecasted order quantity
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE inventory 
        SET suggested_order = %s 
        WHERE product_id = %s
    """, (suggested_order, product_id))
    #make a new SQL column in the stock in ==
    conn.commit()
    conn.close()

    return jsonify({"product_id": product_id, "suggested_order": suggested_order})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\21chu\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
